<a href="https://colab.research.google.com/github/rlagosb/taller_eiv/blob/main/0_Anonimizaci%C3%B3n_Fichas_Programacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracción de Fichas de Programación

En este script extraemos la programación de un conjunto de profesionale de un servicio clínico

In [ ]:
import pandas as pd
import openpyxl as xl
import requests
import os

# Localización de los datos
path_fichas="https://raw.githubusercontent.com/rlagosb/taller_eiv/main/data/programacion/Fichas/"
planilla_fichas = 'Fichas Unidad Endoscopia.xlsx'
# Carpeta de destino
datos_crudos = '/content/datos_crudos/'
if not os.path.exists(datos_crudos): os.makedirs(datos_crudos)

In [ ]:
#@title Descargar Planilla

# Descargar el archivo
response = requests.get(path_fichas + planilla_fichas)
with open(planilla_fichas, 'wb') as f:
    f.write(response.content)

# Cargar planilla
wb = xl.load_workbook(filename = planilla_fichas, data_only=True)

print(f'Planilla {planilla_fichas} descargada ✅')

In [ ]:
#@title Obtener fichas

# Cargar datos de las fichas en las hojas de la planilla
fichas = {}
print('Cargando fichas:')

for ws_name in [name for name in wb.sheetnames if name not in ['Personal', 'Prestaciones', 'Ficha Programación']]:

  #Cargar hoja
  ws=wb[ws_name]

  #obtener datos profesional y eliminar cabecera
  profesional={'rut':ws['C4'].value,'nombre':ws['C3'].value,'profesion':ws['C5'].value,'obs_ficha':ws['F5'].value}
  ws.delete_rows(1,9)
  ws.delete_cols(1,1)

  #obtener actividades y horarios
  prog = (pd.DataFrame(ws.values).rename(columns={0:"ini",1:"fin"}).
          melt(id_vars=['ini','fin'], var_name='dia', value_name='act').
          # eliminar bloques sin actividades programadas
          dropna(subset=['act']))
  prog['dia']=prog['dia']-1

  #agregar datos profesional
  for col in ['rut','nombre','profesion','obs_ficha']:
    prog[col]=profesional[col]

  #agrupar bloques de actividades
  prog['id_prog']=((prog.rut!=prog.rut.shift()) |             # agrupar por cambio de rut
                    (prog.dia!=prog.dia.shift()) |            # por cambio de dia
                    (prog.act!=prog.act.shift())              # por cambio de actividad
                    ).cumsum()                                # asignar correlativo
  fichas[ws_name]=prog.groupby(['id_prog','rut','nombre','profesion','dia','act','obs_ficha'],as_index=False, dropna=False).aggregate({'ini':'min','fin':'max'})

  print(ws.title, '✔')



In [ ]:
#@title Tabular datos

#Consolidar  fichas en una tabla
programacion=pd.concat(fichas, ignore_index=True)

# Calcular duración actividades
programacion['hrs_prog']=(pd.to_datetime(programacion['fin'].astype(str), format='%H:%M:%S') -
                     pd.to_datetime(programacion['ini'].astype(str), format='%H:%M:%S')).dt.total_seconds() / 3600

#Exportar a pm_programacion_medica
programacion.to_excel(datos_crudos+"Programacion.xlsx",sheet_name="PM")
print(programacion.info())
programacion

# Pseudo-anonimización
Eliminamos los datos personales y reemplazamos el identificador

In [ ]:
# Cremos un identificador alternativo

profesionales = programacion[['rut']].drop_duplicates(ignore_index=True).reset_index().rename(columns={'index':'idProfesional'})
profesionales['idProfesional'] +=1
profesionales

In [ ]:
# Agregamos el pseudoidentificador
# y eliminamos los datos personales

programacion_pseudoanonimizada = programacion.merge(profesionales, on='rut', how='left').drop(columns=['rut','nombre','profesion'])
programacion_pseudoanonimizada =

# Anonimización
Eliminamos los campos identificadores y pseudoidentificadores, dejando sólos los campos indispensables

In [ ]:
# Datos Anonimizados

programacion_anonimizada = programacion.groupby(['profesion','act'], dropna=False).hrs_prog.sum().reset_index()
programacion_anonimizada

